In [238]:
# a. set magic 
%load_ext autoreload
%autoreload 2

# b. load packages 
import numpy as np

# c. local modules
from contracept_model_copy import child_model_new
from contracept_model import child_model 
import BI_estimate as estimate
import back as solver

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [239]:
model = child_model()
n = model.n

# Step 2: Initialize value function and policy
T = model.T
V = np.nan + np.zeros([n, T])
V[:, T - 1] = model.life()
pnc1 =  np.nan + np.zeros([n, T])
pnc1[:, T-1] = 1

# Step 3: Backward induction

for t in range(T - 2, -1, -1):
    ev1, pnc = model.bellman(ev0 = V[:, t+1], output=2)
    V[:, t] = ev1  
    pnc1[:, t] = pnc



print(pnc1.round(3))
#print(V.round(2))



[[0.362 0.374 0.387 0.401 0.416 0.432 0.448 0.465 0.482 0.5   0.517 0.534
  0.549 0.563 0.573 0.579 0.581 0.577 0.566 1.   ]
 [0.177 0.188 0.2   0.213 0.227 0.243 0.26  0.278 0.299 0.321 0.345 0.37
  0.398 0.426 0.454 0.482 0.508 0.53  0.546 1.   ]
 [0.118 0.125 0.132 0.141 0.15  0.161 0.173 0.187 0.203 0.22  0.241 0.264
  0.291 0.322 0.356 0.395 0.438 0.483 0.526 1.   ]
 [0.232 0.235 0.238 0.241 0.245 0.249 0.254 0.26  0.267 0.275 0.285 0.296
  0.31  0.327 0.349 0.375 0.408 0.451 0.506 1.   ]
 [0.555 0.555 0.555 0.555 0.555 0.555 0.555 0.555 0.555 0.555 0.555 0.555
  0.555 0.555 0.555 0.555 0.555 0.555 0.555 1.   ]]


In [241]:
data = model.sim_data(pnc1)
data

,id,t,d,x,dx1,x1
0,1,0,0,0,0,0
1,1,1,1,0,0,0
2,1,2,0,0,0,0
3,1,3,1,0,0,0
4,1,4,0,0,0,0
...,...,...,...,...,...,...
19995,1000,15,0,4,0,4
19996,1000,16,0,4,0,4
19997,1000,17,0,4,0,4
19998,1000,18,0,4,1,4


In [217]:
theta = (0.23, -0.15, -0.22)
pnames= ("eta2", "eta3", 'mu')

estimate.ll(theta, model, solver, data, pnames, out=1)

1.225437838378529

In [218]:
dta = model.read_data()

In [219]:
theta = (0.23, -0.15, -0.22)
pnames= ("eta2", "eta3", 'mu')

estimate.ll(theta, model, solver, dta, pnames, out=1)

0.8538839754309433

In [243]:
model = child_model()
# Estimate
model, results, pnames, theta_hat, converged = estimate.estimate(model, solver, data)

# Print 
print(f'Log-Likelihood:     {-results.fun*1000:.4f}')
print(f'mu:                 {theta_hat[0]:.4f}')
#print(f'eta2:               {theta_hat[1]:.4f}')
#print(f'eta3:               {theta_hat[2]:.4f}')


print(model, results, pnames, theta_hat, converged)
print(f'The model converged: {converged}')


/Users/carolinebergholdthansen/Desktop/Dynamic programming/dynamic_cbjk/BI_estimate.py:68: RuntimeWarning: divide by zero encountered in log
  log_lik = np.log(function)                   # Compute log-likelihood-contributions
/Users/carolinebergholdthansen/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_optimize.py:790: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


Log-Likelihood:     -inf
mu:                 0.0000
<contracept_model.child_model object at 0x7fe432d7cd60>  final_simplex: (array([[0.00000000e+00],
       [3.38813179e-24]]), array([inf, inf]))
           fun: inf
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 200
           nit: 67
        status: 1
       success: False
             x: array([0.]) ['mu'] [0.] False
The model converged: False
